# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

file_path_list = []

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list= glob.glob(os.path.join(root,'*'))
    
    #print(file_path_list)

/home/jeffsan/Dropbox/DEND/projects/p2_nosql


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [6]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [7]:
# check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [3]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

In [3]:
session.execute("DROP KEYSPACE IF EXISTS udacity")

#### Create Keyspace

In [4]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [5]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




# Queries:

## 1) Query 1: Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [6]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query1 = "CREATE TABLE IF NOT EXISTS session_history"
query1 = query1 + "(sessionID int, itemInSession int, artist text, song_title text, song_length float,\
                    PRIMARY KEY (sessionID, itemInSession))"
try:
    session.execute(query1)
except Exception as e:
    print(e)
                    

In [7]:
# insert data to table session_history
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_history (sessionID, itemInSession, artist, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5]) ))
        
        

#### Do a SELECT to verify that the data have been inserted into each table

In [8]:
## Add in the SELECT statement to verify the data was entered into the table

query = "SELECT * FROM session_history WHERE sessionID=338 AND itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

columns =  ['artist', 'song_title', 'song_length']
query1_ouput = pd.DataFrame(columns= columns)
    
for row in rows:
    row_list = [[row.artist, row.song_title, row.song_length]]
    query1_ouput = query1_ouput.append(pd.DataFrame(row_list, columns=columns), ignore_index=True)

query1_ouput

,artist,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


The artist, song title and song's length in the music app history that was heard during sessionId=338, and itemInSession=4, is :
- `artist` : Faithless
- `song_title`: Music Matters (Mark Knight Dub)
- `song_length`: 495.307312

## 2) Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [34]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query2 = "CREATE TABLE IF NOT EXISTS user_history"
query2 = query2 + "(userID int, sessionID int, itemInSession int, artist text, song_title text, first_name text, last_name text,\
                    PRIMARY KEY ( (userID, sessionID), itemInSession))"
try:
    session.execute(query2)
except Exception as e:
    print(e)
                    

In [35]:
# Insert data to user_history table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_history (userID, sessionId, itemInSession, artist, song_title, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]  ))
        
        

In [49]:
## Add in the SELECT statement to verify the data was entered into the table

query = "SELECT * FROM user_history WHERE userID=10 AND sessionID=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

columns = ['artist', 'song_title', 'first_name', 'last_name']
query2_ouput = pd.DataFrame(columns=columns)
    
for row in rows:
    row_list = [[row.artist, row.song_title, row.first_name, row.last_name]]
    query2_ouput = query2_ouput.append(pd.DataFrame(row_list, columns=columns), ignore_index=True)



The name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182, are showed in below table.

In [50]:
query2_ouput

,artist,song_title,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


# 3) Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own

In [42]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query3 = "CREATE TABLE IF NOT EXISTS song_history"
query3 = query3 + "(song_title text, userID int, first_name text, last_name text,\
                    PRIMARY KEY (song_title, userID))"
try:
    session.execute(query3)
except Exception as e:
    print(e)
                    
                    

In [43]:
# insert data to song_history table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_history (song_title, userID, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))
        
        

In [52]:
## Add in the SELECT statement to verify the data was entered into the table

query = "SELECT * FROM song_history WHERE song_title='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
columns = ['first_name', 'last_name']
query3_ouput = pd.DataFrame(columns=columns)
    
for row in rows:
    row_list = [[row.first_name, row.last_name]]
    query3_ouput = query3_ouput.append(pd.DataFrame(row_list, columns=columns), ignore_index=True)
    

Every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own, are Jacqueline Lynch, Tegan Levine, and Sara Johnson.

In [53]:
query3_ouput

,first_name,last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [54]:
for table in ['session_history', 'user_history', 'song_history']:
    query = f"DROP TABLE IF EXISTS {table}"
    try:
        rows = session.execute(query)
    except Exception as e:
        print(table, e)


### Close the session and cluster connection¶

In [9]:
session.shutdown()
cluster.shutdown()